# Tools

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///Chinook.db")

In [2]:
from langchain.chat_models import init_chat_model

model = init_chat_model("openai:gpt-5")

In [3]:
from langchain_core.tools import tool
    
@tool
def execute_sql(query: str) -> str:
    """Execute a SQLite command and return results."""
    try:
        return db.run(query)
    except Exception as e:
        return f"Error: {e}"


In [4]:
from langchain_core.tools import tool
import requests

FALLBACK_RATES = {"USD": 1.0, "EUR": 0.92, "JPY": 150.0, "GBP": 0.79}

@tool
def convert_currency(amount: float, to_currency: str) -> float:
    """Convert an amount in USD to another currency using live exchange rates.
    Always use this tool if the user requests a currency different 
    from the one stored in the database (USD)."""
    try:
        url = f"https://api.exchangerate.host/convert?from=USD&to={to_currency}&amount={amount}"
        response = requests.get(url, timeout=5)
        response.raise_for_status()
        data = response.json()
        result = data.get("result")
        if result is not None:
            return round(float(result), 2)
    except Exception as e:
        print(f"[convert_currency] Falling back due to error: {e}")

    if to_currency not in FALLBACK_RATES:
        raise ValueError("Unsupported currency in fallback mode")
    return round(amount * FALLBACK_RATES[to_currency], 2)

In [5]:
SYSTEM = f"""You are a careful SQLite analyst.

Rules:
- Think step-by-step.
- When you need data, call the tool `execute_sql` with ONE SELECT query.
- Read-only only; no INSERT/UPDATE/DELETE/ALTER/DROP/CREATE/REPLACE/TRUNCATE.
- Limit to 5 rows unless the user explicitly asks otherwise.
- If the tool returns 'Error:', revise the SQL and try again.
- Prefer explicit column lists; avoid SELECT *.
"""

In [6]:
from langchain.agents import create_agent
from langchain_core.messages import SystemMessage

llm = init_chat_model("gpt-5", model_provider="openai")

agent = create_agent(
    model=llm,
    tools=[execute_sql, convert_currency],
    prompt=SystemMessage(content=SYSTEM),
)

In [7]:
question = "What is the most costly purchase by Frank Harris in EUR?"
steps=[]

for step in agent.stream(
    {"messages": [{"role": "user", "content": question}]},
    stream_mode="values",
):
    step["messages"][-1].pretty_print()
    steps.append(step)

================================ Human Message =================================

What is the most costly purchase by Frank Harris in EUR?
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_763zHvExCCeLE2IDJsB1LeZS)
 Call ID: call_763zHvExCCeLE2IDJsB1LeZS
  Args:
    query: SELECT id, first_name, last_name FROM customers WHERE (first_name || ' ' || last_name) = 'Frank Harris' LIMIT 5;
================================= Tool Message =================================
Name: execute_sql

Error: (sqlite3.OperationalError) no such table: customers
[SQL: SELECT id, first_name, last_name FROM customers WHERE (first_name || ' ' || last_name) = 'Frank Harris' LIMIT 5;]
(Background on this error at: https://sqlalche.me/e/20/e3q8)
================================== Ai Message ==================================
Tool Calls:
  execute_sql (call_kkp7QVUXGeuSjhUowmHIVI9q)
 Call ID: call_kkp7QVUXGeuSjhUowmHIVI9q
  Args:
    query: SELECT name 